In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="MONK2"
JobNameAbbrev = "M2"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.015],
                       "Type": ["Classification"]}

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

In [3]:
ParameterVector.shape

(200, 11)

# Include/exclude Random Forest Simulations

In [4]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.00],
                       "rashomon_bound_adder": [0],
                       "Type": ["Classification"]}
RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [5]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [6]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,MONK2,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.015,Classification,0M2_DPL0.015,MONK2/TreeFarms/Raw/0M2_DPL0.015.pkl
1,MONK2,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.015,Classification,0M2_UNQ0.015,MONK2/TreeFarms/Raw/0M2_UNQ0.015.pkl
2,MONK2,0,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,0M2_RF0.0,MONK2/RandomForestClassification/Raw/0M2_RF0.0...
3,MONK2,1,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,1M2_RF0.0,MONK2/RandomForestClassification/Raw/1M2_RF0.0...
4,MONK2,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.015,Classification,1M2_UNQ0.015,MONK2/TreeFarms/Raw/1M2_UNQ0.015.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,MONK2,98,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,98M2_RF0.0,MONK2/RandomForestClassification/Raw/98M2_RF0....
296,MONK2,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.015,Classification,98M2_UNQ0.015,MONK2/TreeFarms/Raw/98M2_UNQ0.015.pkl
297,MONK2,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.015,Classification,99M2_DPL0.015,MONK2/TreeFarms/Raw/99M2_DPL0.015.pkl
298,MONK2,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.015,Classification,99M2_UNQ0.015,MONK2/TreeFarms/Raw/99M2_UNQ0.015.pkl


In [7]:
# ParameterVector.loc[len(ParameterVector)+0] = Duplicate4
# ParameterVector.loc[len(ParameterVector)+1] = Unique15
# ParameterVector.loc[len(ParameterVector)+2] = Duplicate68
# ParameterVector.loc[len(ParameterVector)+3] = Unique68

In [8]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

In [ ]:
# ### 4 ###
# Duplicate4 = {"Data" : "BreastCancer",
# "Seed" : 4,
# "TestProportion" : 0.2,
# "CandidateProportion" : 0.8,
# "SelectorType" : "TreeEnsembleQBCFunction",
# "ModelType" : "TreeFarmsFunction",
# "UniqueErrorsInput" : 0,
# "n_estimators" : 100,
# "regularization" : 0.01,
# "rashomon_bound_adder" : 0.009,
# "Type" : "Classification",
# "JobName" : "4BC_DPL0.009",
# "Output" : "BreastCancer/TreeFarms/Raw/4BC_DPL0.009.pkl"}


# ### Add New Values ###
# # ParameterVector.loc[len(ParameterVector)+0] = Duplicate4

# ### Save Again ###
# output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
# ParameterVector.to_csv(output_path, index=False)